In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import time
import requests
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('./fires_in_ph.csv')

C:\Users\jackv\AppData\Local\Temp\ipykernel_12784\2703195330.py:1: DtypeWarning: Columns (99,104,109) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./fires_in_ph.csv')


In [3]:
from scipy.stats import pearsonr

corrs = []
public_pa_ny = df
# Iterate over numerical columns in pr
for col in df.select_dtypes(include='number').columns:
    if col not in ['all_fires', 'block2kx', 'bg2kx', 'place_inc2kx', 
                   'hlc', 'dpvrc', 'std_zip9', 'dpbc', 'dpbc_cksum', 'std_zip11', 
                   'c1pprb', 'blkgrp_level', 'exec_dir_fax', 
                   'national_bldg_id', 'dpvact', 'dpvnost', 'place2kx',
                   'dpv', 'zcta2kx', 'place_level', 'x', 'y',
                   'state2kx', 'cnty2kx', 'tract2kx', 'curcnty', 'curcosub', 'msa',
                   'cbsa', 'necta', 'lat', 'lon', 'county_level',
                   'tract_level', 'ha_fax_num', 'exec_dir_phone', 'house+cooking_fires_15-19',
                    'house+cooking_fires_20-21', 'ha_phn_num']:
        df['micro'] = df.micro.fillna(df.micro.mean())
        df['metro'] = df.micro.fillna(df.micro.mean())
        df['annl_expns_amnt_prev_yr'] = df.annl_expns_amnt_prev_yr.fillna(df.annl_expns_amnt_prev_yr.mean())
        df['std_zip5'] = df.std_zip5.fillna(-1)
        df['pha_total_units'] = df.pha_total_units.fillna(df.pha_total_units.mean())

        # Get correlation and p-value
        correlation, p_value = pearsonr(df[col], df['building_fires_11_19'])
        correlation2, p_value2 = pearsonr(df[col], df['building_fires_20_21'])

        corrs.append((col, correlation, p_value, correlation2, p_value2))

corrs_df = pd.DataFrame(corrs, columns=['columns', 'corr', 'p_value', 'corr2', 'p_value2'])
corrs_df['significant1'] = corrs_df['p_value'] < 0.05
corrs_df['significant2'] = corrs_df['p_value2'] < 0.05

In [4]:
pd.set_option('display.max_row', 200)
corrs_df[(corrs_df.significant1)|(corrs_df.significant2)].sort_values('corr')

,columns,corr,p_value,corr2,p_value2,significant1,significant2
64,std_zip5,-0.029856,4.224570e-39,-0.017403,2.464008e-14,True,True
63,micro,-0.012225,8.536863e-08,-0.009588,2.669673e-05,True,True
62,metro,-0.012225,8.536863e-08,-0.009588,2.669673e-05,True,True
67,pha_total_units,-0.005179,2.328744e-02,-0.003475,1.279051e-01,True,False
52,pct_bed3,0.017241,4.256355e-14,0.015277,2.197647e-11,True,True
32,pct_female_head_child,0.021557,3.581862e-21,0.018319,1.012084e-15,True,True
51,pct_bed2,0.021564,3.489301e-21,0.017245,4.195201e-14,True,True
30,pct_1adult,0.021690,2.049272e-21,0.018542,4.551489e-16,True,True
53,pct_overhoused,0.022580,4.491682e-23,0.018823,1.636836e-16,True,True
57,chldrn_mbr_cnt,0.030425,1.535343e-40,0.024088,4.922862e-26,True,True


There are some public housing buildings that share an address, even though they are separate structures. For this analysis, in that case, I will keep only the first record. However, future work should be done to see if improved matching is possible.

In [5]:
df['total_fires'] = df['building_fires_11_19'] + df['building_fires_20_21']

## Do some buildings stand out as being the worst?

In [6]:
df['fires_adj'] = df.total_fires / df.total_units

Most buildings don't have any fires from 2011-2021.

In [7]:
len(df[df.total_fires == 0]) / len(df)

0.984907626964067

But those that do usually have more than one fire in that time period.

In [8]:
df[df['total_fires'] > 0]['total_fires'].mean()

2.489986187845304

We can also look at fires per unit

In [9]:
df[df['fires_adj'] > 0]['fires_adj'].mean()

0.4428342751237843

88 buildings in our dataset have had, on average, at least one fire a year for the last 10 years.

In [10]:
len(df[df.total_fires > 10])

88

These buildings have had about 1.6 fires per unit. 

In [11]:
df[df.total_fires > 10].fires_adj.mean()

1.603829802132413

These buildings are mostly elevator-structure buildings.

In [12]:
df[df.total_fires > 10].groupby('building_type_code').size()

building_type_code
ES    76
RW     3
SF     1
WU     8
dtype: int64

In [13]:
print(df[df.total_fires > 0].total_units.describe())
print()
print(df[df.total_fires == 0].total_units.describe())

count    2896.000000
mean       35.975138
std        56.087676
min         1.000000
25%         2.000000
50%         6.000000
75%        54.000000
max       464.000000
Name: total_units, dtype: float64

count    188989.000000
mean          4.618486
std          13.256085
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max         477.000000
Name: total_units, dtype: float64


[Link with info](https://www.hud.gov/sites/documents/DOC_11633.PDF) on building code abbreviations.

* SF -> Single Family / Detached
* RW -> Row/Townhouse Dwelling
* WU -> Walk-UP/MultiFamily Apartment
* ES -> Elevator Structure 
* SD -> Semi-Detatched

In [14]:
df.building_type_code = df.building_type_code.str.upper()

In [15]:
(df.groupby('building_type_code').size() / len(df)).sort_values(ascending=False)

building_type_code
SD     0.380374
RW     0.302958
SF     0.203773
WU     0.091492
ES     0.021357
NDS    0.000047
dtype: float64

In [16]:
df.groupby('building_type_code').total_fires.mean().sort_values(ascending=False)

building_type_code
ES     1.097365
WU     0.051606
SF     0.012992
RW     0.012850
SD     0.007577
NDS    0.000000
Name: total_fires, dtype: float64

In [17]:
df.groupby('building_type_code').fires_adj.mean().sort_values(ascending=False)

building_type_code
SF     0.012787
ES     0.012128
WU     0.011893
RW     0.004099
SD     0.003914
NDS    0.000000
Name: fires_adj, dtype: float64

S is house/street and H is highrise

In [18]:
df.groupby('addr_type').size().sort_values(ascending=False)

addr_type
S    137689
H     41962
P       289
R        46
G         3
F         2
dtype: int64

In [19]:
df.groupby('addr_type').total_fires.mean().sort_values(ascending=False)

addr_type
F    2.000000
H    0.130999
S    0.012419
G    0.000000
P    0.000000
R    0.000000
Name: total_fires, dtype: float64

In [20]:
df.groupby('addr_type').fires_adj.mean().sort_values(ascending=False)

addr_type
F    0.023529
S    0.007830
H    0.004868
G    0.000000
P    0.000000
R    0.000000
Name: fires_adj, dtype: float64

##### !!!!!!!!!! Even though a building being large is a good predictor that it will have at least one fire, most large buildings have zero matched fires! So identifying high risk properties within ES structures might be helpful. However, we might need to grade the classifier differntly. For example, it might be great at identifying all ES properties as high risk, but then doing nothing else to diferentiate them.

In [21]:
df[df.building_type_code == 'ES'].total_fires.describe()

count    4098.000000
mean        1.097365
std         5.008271
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       143.000000
Name: total_fires, dtype: float64

In [22]:
df[df.addr_type == 'H'].total_fires.describe()

count    41962.000000
mean         0.130999
std          1.644532
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        143.000000
Name: total_fires, dtype: float64

## Do buildings with fires differ in terms of year-built?

In [23]:
df['construction_year'] = pd.to_datetime(df['construct_date']).dt.year
df['construction_month'] = pd.to_datetime(df['construct_date']).dt.month_name()
df.loc[df['construction_year'] > 2023, 'construction_year'] = np.nan

In [24]:
df[df.total_fires == 0].construction_year.describe()

count    149228.000000
mean       1972.333912
std          17.518137
min        1791.000000
25%        1962.000000
50%        1971.000000
75%        1982.000000
max        2022.000000
Name: construction_year, dtype: float64

In [25]:
df[df.total_fires > 0].construction_year.describe()

count    2312.000000
mean     1974.385381
std        18.510124
min      1900.000000
25%      1965.000000
50%      1972.000000
75%      1983.000000
max      2022.000000
Name: construction_year, dtype: float64

In [26]:
df[df.construction_year >= df.construction_year.median()].total_fires.mean()

0.03824301802939246

In [27]:
df[df.construction_year < df.construction_year.median()].total_fires.mean()

0.035293629727936295

## Base rates of different states

Fires per public housing building.

In [28]:
(
df.groupby('std_st').total_fires.sum() / df.groupby('std_st').size()
 ).sort_values(ascending=False)

std_st
MA    0.372135
NJ    0.275563
RI    0.168632
DC    0.150888
PA    0.106865
CT    0.104405
WA    0.082998
NH    0.079545
ME    0.073214
NE    0.068817
MI    0.068755
SD    0.062500
IL    0.057474
NY    0.052549
ID    0.049645
KY    0.048760
DE    0.043876
MN    0.040796
MD    0.039080
UT    0.038062
NV    0.036145
OH    0.035841
AK    0.031429
SC    0.029841
VA    0.029632
WI    0.025692
CO    0.023969
ND    0.021858
WV    0.019337
WY    0.018939
TN    0.018546
GA    0.017811
KS    0.017787
IN    0.016843
NC    0.014332
IA    0.013564
MO    0.013430
HI    0.013021
OK    0.010616
AZ    0.010511
CA    0.010348
FL    0.009263
MS    0.008786
MT    0.007653
LA    0.006658
OR    0.006323
AL    0.004971
TX    0.004285
AR    0.003327
NM    0.002303
PR    0.000000
GU    0.000000
VI    0.000000
VT    0.000000
dtype: float64

In [29]:
(
df.groupby('std_st').fires_adj.mean()
 ).sort_values(ascending=False)

std_st
MA    0.049298
ID    0.021630
PA    0.013696
CT    0.012992
SC    0.012095
MI    0.011936
NE    0.011722
DE    0.011613
SD    0.011530
OH    0.011068
WI    0.010937
MD    0.010826
MN    0.010580
NV    0.010237
RI    0.010141
DC    0.009278
NJ    0.009183
UT    0.009072
WA    0.007998
IL    0.007967
NH    0.007879
GA    0.006840
TN    0.006602
NC    0.006336
KY    0.006248
KS    0.006096
MO    0.005790
CO    0.005609
ND    0.005539
ME    0.005255
AZ    0.005147
IN    0.005081
MS    0.004824
VA    0.004716
AK    0.004348
NY    0.004187
FL    0.003938
IA    0.003827
LA    0.003559
WV    0.003403
OR    0.003368
OK    0.003125
MT    0.002551
AL    0.002355
NM    0.002303
CA    0.002183
WY    0.002112
AR    0.001551
TX    0.001392
HI    0.001227
PR    0.000000
VI    0.000000
VT    0.000000
GU    0.000000
Name: fires_adj, dtype: float64

## Check the damages from the identified fires

In [30]:
tuples = df[
    (df.building_fires_11_19 > 0) |
    (df.building_fires_20_21 > 0)
]
tuples = tuples[['std_addr', 'std_city', 'std_st', 'std_zip5']].copy()

In [ ]:
# 21 minutes to run
tuples['address'] = tuples[['std_addr', 'std_city', 'std_st', 'streettype', 'streetsuf', 'apt_no']] \
    .apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [31]:
conn = sqlite3.Connection('./data/nfirs/fire_data.db')
query = """
SELECT *
FROM incident_address ia
WHERE (ia.NUM_MILE || ' ' || COALESCE(ia.STREET_PRE, '') || ' ' || ia.STREETNAME || ' ' || COALESCE(ia.STREETTYPE, '') || ' ' || COALESCE(ia.STREETSUF, '') || ' ' || COALESCE(ia.APT_NO, '') || ' ' || ia.CITY || ' ' || ia.STATE || ' ' || ia.ZIP5) = ?
"""


matches = []
for i, t in enumerate(tuples):
    if i % 100 == 0:
        print(i)
    result = conn.execute(query, (t,)).fetchall()
    if result:
        matches.extend(result)

conn.close()

0


In [32]:
matches

[]

## Manual inspection sanity check

For the worst properties, and UT, and some random properties, manually check the NFIRS database and the public housing dataset.

## Look at all highrise buildings where a fire was identified, and then pull the full info from FEMA

## Summarize death, injury, and property loss, and how buildings with lots of that differ